## 之前的是直接写死的数据，但我们经常会从本地读入数据

In [5]:
import numpy as np
import operator

In [146]:
def classify(testX,train,labeks,k):
    """
    testX:测试数据
    train：训练数据
    labels：训练数据的标签
    k：和测试数据最相近k个点
    """
    dataSetSizes = train.shape[0]
    
    """
    这里现在需要做的就是找出测试数据点和训练样本点之间的距离
    根据训练样本点的大小来复制
    """
    diffMat = np.tile(testX,(dataSetSizes,1)) - train
    sqDiffMat = diffMat**2 
    sqDistance = sqDiffMat.sum(axis=1)
    distance = sqDistance ** 0.5
    sortIndex = distance.argsort()
    
    classCount = {}
    for i in range(k):
        index = sortIndex[i]
        voteLabel = labels[index]
        classCount[voteLabel] = classCount.get(voteLabel,0) + 1
        
    finalTarget = sorted(classCount.iteritems(),key=operator.itemgetter(1),reverse=True)
    return  finalTarget[0][0]

In [147]:
def file2Matrix(fileName):
    """
    从文件读取训练数据，并存储为矩阵
    """
    returnMat   = []
    classLabelVector = []
    with open(fileName) as ifile:  
        for line in ifile: 
            tokens = line.strip().split(' ')
            returnMat.append([float(tk) for tk in tokens[:-1]])  
            classLabelVector.append(tokens[-1])
    return np.array(returnMat),np.array(classLabelVector)

In [137]:
#在这里进行归一化的目的就是防止某些数据点差异太大影响预测的结果
def autoNorm(dataSet):
    """
    把训练的数据都归一化，归一化的方式才用的极差归一化控制在[0,1]的范围内
    公式：m - min /  max - min m就是矩阵
    [1]获取每列的最大值1和最小值，求得两个之差
    [2]创建存储归一化数据的矩阵，大小和原矩阵一样
    [3]计算每个元素的归一化
    """
    minVals = dataSet.min(0)
    maxVals = dataSet.max(0)
    ranges = maxVals - minVals
    
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    normDataSet = dataSet - np.tile(minVals,(m,1))
    normDataSet = normDataSet/np.tile(ranges,(m,1))
    
    return normDataSet, ranges, minVals

In [182]:
def startClass():
    """
    ration：取总数据的20%作为训练数据
    """
    ration = 0.20  
    datingDataMat,datingLabels = file2Matrix("dataTrain.txt")       #load data setfrom file  
    normMat, ranges, minVals = autoNorm(datingDataMat)  
    m = normMat.shape[0]  
    numTestVecs = int(m*ration) 
    errorCount = 0.0 
    for i in range(numTestVecs):
        classifierResult = classify(normMat[i,:],normMat[numTestVecs:m,:],datingLabels[numTestVecs:m],4)
        if (classifierResult != datingLabels[i]): 
            errorCount += 1.0  
            
    print "the total error rate is: %f" % (errorCount/float(numTestVecs))  

In [183]:
startClass()

the total error rate is: 0.666667
